# On va considérer que tu dois faire un formulaire avec un titre et le texte

In [1]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: click in c:\users\benjamin\anaconda3\lib\site-packages (from nltk>=3.2.1->newspaper3k) (7.1.2)



In [2]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install BeautifulSoup

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Benjamin\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Benjamin\\AppData\\Local\\Temp\\pip-install-ueb7w0gw\\BeautifulSoup\\setup.py'"'"'; __file__='"'"'C:\\Users\\Benjamin\\AppData\\Local\\Temp\\pip-install-ueb7w0gw\\BeautifulSoup\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Benjamin\AppData\Local\Temp\pip-pip-egg-info-1bwcvgr3'
         cwd: C:\Users\Benjamin\AppData\Local\Temp\pip-install-ueb7w0gw\BeautifulSoup\
    Complete output (6 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\Benjamin\AppData\Local\Temp\pip-install-ueb7w0gw\BeautifulSoup\setup.py", line 3
        "You're trying to run a very old release of Beautiful Soup under Python 3. Thi


  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from newspaper import Article
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score

In [12]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords=set(stopwords.words('english'))
from textblob import TextBlob #for subjectivity and polarity purpose

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def tokenizetext(text):
    return word_tokenize(text)
def words(text):
    l = [word for word in word_tokenize(text) if word.isalpha()]
    return l
def unique_words(text):
    return list(set(words(text)))
def rate_uni_words(text):
    uni_words = len(unique_words(text))/len(words(text))
    return uni_words
def avglengthtoken(text):
    w = words(text)
    sum = 0
    for item in w:
        sum+=len(item)
    avglen = sum/len(w)
    return avglen
def n_non_stop_unique_tokens(text):
    uw = unique_words(text)
    n_uw = [item for item in uw if item not in stopwords]
    w = words(text)
    n_w = [item for item in w if item not in stopwords]
    rate_nsut = len(n_uw)/len(n_w)
    return rate_nsut
def numlinks(article):
    return len(BeautifulSoup(article).findAll('link'))
def get_subjectivity(a_text):
    return a_text.sentiment.subjectivity
def get_polarity(a_text):
    return a_text.sentiment.polarity
def word_polarity(words):
    pos_words = []
    ppos_words = [] # polarity of pos words
    neg_words = []
    pneg_words = [] # polarity of negative words
    neu_words = []
    pneu_words = [] # polarity of neutral words
    for w in words:
        an_word = TextBlob(w)
        val = an_word.sentiment.polarity
        if val > 0:
            pos_words.append(w)
            ppos_words.append(val)
        if val < 0:
            neg_words.append(w)
            pneg_words.append(val)
        if val == 0 :
            neu_words.append(w)
            pneu_words.append(val)
    return pos_words,ppos_words,neg_words,pneg_words,neu_words,pneu_words
def avg_pol_pw(text):    
    totalwords = words(text)
    res = word_polarity(totalwords)
    return np.sum(res[1])/len(res[0])
def avg_pol_nw(text):    
    totalwords = words(text)
    res = word_polarity(totalwords)
    return np.sum(res[3])/len(res[2])

In [14]:
import nltk
nltk.download('punkt')
def masterDF(titre, texte, categ):
    finrows = []
    row = {}
    row['n_tokens_title'] = len(words(titre))
    row['n_tokens_content'] = len(words(texte))
    row['n_unique_tokens'] = len(unique_words(texte))
    row['average_token_length'] = avglengthtoken(texte)
    row['n_non_stop_unique_tokens'] = n_non_stop_unique_tokens(texte)
    row['num_hrefs'] = numlinks(texte)
    
    analysed_text = TextBlob(texte)
    row['global_subjectivity'] = get_subjectivity(analysed_text)
    row['avg_positive_polarity'] = avg_pol_pw(texte)
    row['global_sentiment_polarity'] = get_polarity(analysed_text)
    finrows.append(row)
    
    liste = ["data_channel_is_world", "data_channel_is_tech", "data_channel_is_socmed", "data_channel_is_bus", "data_channel_is_entertainment", "data_channel_is_lifestyle"]
    row["data_channel_is_world"] =0
    row["data_channel_is_tech"]=0
    row["data_channel_is_socmed"]=0
    row["data_channel_is_bus"]=0
    row["data_channel_is_entertainment"]=0
    row["data_channel_is_lifestyle"]=0
    row[liste[categ]] = 1
    
    
    return finrows

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
import pickle 
xgb2 = pickle.load(open("xgbmodel2", 'rb'))

# En dessous t'as juste à mettre ton titre et ton texte dans la fonction

In [16]:
def Prediction(titre, texte, categ):
    finrows = masterDF(titre, texte, categ)
    masterdf = pd.DataFrame(finrows, columns = ['n_tokens_title','n_tokens_content','n_unique_tokens','average_token_length','n_non_stop_unique_tokens','num_hrefs','global_subjectivity',
                                   'avg_positive_polarity','global_sentiment_polarity',
    'data_channel_is_world',"data_channel_is_tech", "data_channel_is_socmed","data_channel_is_bus","data_channel_is_entertainment","data_channel_is_lifestyle"])
    #0 world / 1 tech / 2 social / 3 business / 4 entertainement / 5 lifestyle
    pred = xgb2.predict(masterdf)
    pop = 'Popular' if pred == 1 else 'Unpopular'
    print("Predicted popularity :",pop)

# Exemple

In [18]:
Prediction("Emmanuel Macron: French president tests positive for Covid",
           "France's President Emmanuel Macron has tested positive for Covid-19 after developing symptoms.\
The 42 year old will now self-isolate for seven days, the Elysée Palace said in a statement.\
Mr Macron is still in charge of running the country and will work remotely, said an official.\
EU chief Charles Michel and Spanish Prime Minister Pedro Sánchez are both self-isolating after coming into contact with Mr Macron on Monday.\
France this week imposed an overnight curfew to help deal with soaring cases there.\
There have been two million confirmed cases in the country since the epidemic began, with more than 59,400 deaths, according to data from Johns Hopkins University.\
    How are France and other European countries tackling the pandemic?\
    French culture takes centre stage in Covid protest\
    The meteoric rise of France's youngest president\
Who has Macron had contact with?\
The President of the Republic has been diagnosed positive for Covid-19 today, the Elysée said on Thursday morning. The diagnosis was made following a test performed at the onset of the first symptoms, the statement added.\
It is not yet known how Mr Macron caught the virus but his office said it was identifying any close contacts he has made in recent days.\
Prime Minister Jean Castex, 55, and Parliament Speaker Richard Ferrand, 58, are both self-isolating, their offices confirmed.\
Mr Castex, who is not showing any symptoms, was due to introduce the government's Covid vaccination policy in the Senate on Thursday - now Health Minister Olivier Véran is doing it instead. ",
          0)
#0 world / 1 tech / 2 social / 3 business / 4 entertainement / 5 lifestyle


Predicted popularity : Popular
